In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [2]:
full_targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")
full_mtl = Table.read("strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits")

ii = np.in1d(full_mtl['TARGETID'], zcat['TARGETID'])
in_mtl = full_mtl[ii]

ii = np.in1d(full_targets['TARGETID'], zcat['TARGETID'])
in_targets = full_targets[ii]

print(len(zcat), len(in_mtl), len(in_targets))
in_mtl.sort(keys='TARGETID')
zcat.sort(keys='TARGETID')
in_targets.sort(keys='TARGETID')


1023458 1023458 1023458


In [3]:
assert np.all(in_mtl['TARGETID']==zcat['TARGETID'])
assert np.all(in_mtl['TARGETID']==in_targets['TARGETID'])

In [4]:
zcat_large = zcat[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_mtl_large = in_mtl[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_targets_large = in_targets[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

1060 0.10357044451262289


In [5]:
names = {}
for i in range(len(in_mtl_large)):
    a = desi_mask.names(in_mtl_large['DESI_TARGET'][i])
    n = ''
    for aa in a:
        n = n +' '+ aa
    if n not in names.keys():
        names[n] = 1
    else:
        names[n] += 1

In [6]:
names

{' ELG QSO ELG_SOUTH QSO_SOUTH': 603,
 ' QSO QSO_SOUTH BGS_ANY MWS_ANY': 72,
 ' QSO QSO_SOUTH BGS_ANY': 41,
 ' QSO QSO_SOUTH MWS_ANY': 8,
 ' QSO QSO_SOUTH': 320,
 ' ELG QSO ELG_SOUTH QSO_SOUTH BGS_ANY': 11,
 ' LRG LRG_2PASS LRG_SOUTH LRG_2PASS_SOUTH': 1,
 ' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY': 3,
 ' QSO QSO_SOUTH STD_FAINT MWS_ANY': 1}

In [7]:
zcat_large[:10]

TARGETID,BRICKNAME,SPECTYPE,Z,ZERR,ZWARN,NUMOBS
int64,str8,str10,float32,float32,int32,int32
35185796730325505,,QSO,3.0,0.0,0,5
35185796747102440,,QSO,3.0,0.0,0,5
35185796789044894,,QSO,3.0,0.0,0,5
35185796789047055,,QSO,3.0,0.0,0,5
35185796851960484,,QSO,3.0,0.0,0,5
35185796868739057,,QSO,3.0,0.0,0,5
35185796881321222,,QSO,3.0,0.0,0,5
35185796885515606,,QSO,3.0,0.0,0,5
35185796898098931,,QSO,3.0,0.0,0,5


In [8]:
in_mtl_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.489546376,1.49643925801,35185796730325505,393222,0,0,3400,0.278426456519,4,27057,1,3500,3
164.436610671,1.48944345901,35185796747102440,3458764513820803076,262148,1280,3400,0.933516791912,4,27060,1,3500,7
166.899619394,1.45612873377,35185796789044894,393222,0,0,3400,0.670585445083,4,27037,1,3500,3
166.839541836,1.56063350785,35185796789047055,393222,0,0,3400,0.91003790398,4,27037,1,3500,3
170.707188463,1.51110422127,35185796851960484,393222,0,0,3400,0.797581941387,4,26990,1,3500,3
171.652835438,1.58829568805,35185796868739057,393222,0,0,3400,0.834028952502,4,26991,1,3500,3
172.490248608,1.57112594959,35185796881321222,1152921504607109124,262148,0,3400,0.549945419568,4,26989,2,3500,5
172.540803468,1.5580209201,35185796885515606,2305843009213956100,0,1280,3400,0.569529570617,4,27000,1,3500,7
173.298690723,1.58706829616,35185796898098931,393222,0,0,3400,0.564591717252,4,27000,1,3500,3


In [9]:
in_targets_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.489546376,1.49643925801,35185796730325505,393222,0,0,3400,0.278426456519,4,27057,4,3400,3
164.436610671,1.48944345901,35185796747102440,3458764513820803076,262148,1280,3400,0.933516791912,4,27060,4,3400,7
166.899619394,1.45612873377,35185796789044894,393222,0,0,3400,0.670585445083,4,27037,4,3400,3
166.839541836,1.56063350785,35185796789047055,393222,0,0,3400,0.91003790398,4,27037,4,3400,3
170.707188463,1.51110422127,35185796851960484,393222,0,0,3400,0.797581941387,4,26990,4,3400,3
171.652835438,1.58829568805,35185796868739057,393222,0,0,3400,0.834028952502,4,26991,4,3400,3
172.490248608,1.57112594959,35185796881321222,1152921504607109124,262148,0,3400,0.549945419568,4,26989,4,3400,5
172.540803468,1.5580209201,35185796885515606,2305843009213956100,0,1280,3400,0.569529570617,4,27000,4,3400,7
173.298690723,1.58706829616,35185796898098931,393222,0,0,3400,0.564591717252,4,27000,4,3400,3
